In [1]:
import sys
sys.path.insert(0, "..")

from laboneq.simple import *

from tests.helpers.device_setups import single_tunable_transmon_setup, single_tunable_transmon_qubits

from laboneq_applications.core.quantum_operations import create_pulse
from laboneq_applications.qpu_types.tunable_transmon import TunableTransmonQubit
from laboneq.dsl.parameter import SweepParameter
import numpy as np

from laboneq_applications import dsl
from laboneq_applications.core.build_experiment import qubit_experiment
from laboneq_applications.core.quantum_operations import QuantumOperations
from laboneq_applications.qpu_types.tunable_transmon import (
    TunableTransmonOperations,
    TunableTransmonQubit
)
from laboneq_applications.workflow import task

In [7]:
# setup is an ordinary LabOne Q DeviceSetup:
setup = single_tunable_transmon_setup()
# qubits is a list of LabOne Q Application Library TunableTransmonQubit qubits:
qubits = single_tunable_transmon_qubits(setup)

In [8]:
def myx180(q : TunableTransmonQubit, 
           number_of_pulses : int | SweepParameter,
           transition : str | None = 'ge'
           ):
    """Plays N times x180-like pulses with certain calibration """
    
    drive_line, param = q.transition_parameters(transition)
    
    pulse = create_pulse(param["pulse"], name="my-own-pulse") # no ovverrides
    
    if (isinstance(number_of_pulses, int)):
        for i in range(number_of_pulses):
            dsl.play(
                q.signals["drive"],
                pulse=pulse,
                amplitude=param["amplitude_pi"],
                length=param["length"],
            )
        else:
            with dsl.match(sweep_parameter = number_of_pulses):
                for npulse in number_of_pulses.values():
                    with dsl.case(npulse):
                        for i in range(npulse):
                            dsl.play(
                                q.signals["drive"],
                                pulse=pulse,
                                amplitude=param["amplitude_pi"],
                                length=param["length"],
                            )
                            

In [9]:
qop = TunableTransmonOperations()
# substitute it in the quantum operation
qop["x180"] = myx180

print(qop.x180.src)

def myx180(q : TunableTransmonQubit, 
           number_of_pulses : int | SweepParameter,
           transition : str | None = 'ge'
           ):
    """Plays N times x180-like pulses with certain calibration """

    drive_line, param = q.transition_parameters(transition)

    pulse = create_pulse(param["pulse"], name="my-own-pulse") # no ovverrides

    if (isinstance(number_of_pulses, int)):
        for i in range(number_of_pulses):
            dsl.play(
                q.signals["drive"],
                pulse=pulse,
                amplitude=param["amplitude_pi"],
                length=param["length"],
            )
        else:
            with dsl.match(sweep_parameter = number_of_pulses):
                for npulse in number_of_pulses.values():
                    with dsl.case(npulse):
                        for i in range(npulse):
                            dsl.play(
                                q.signals["drive"],
                                pulse=pulse,
         

In [10]:
@task
@qubit_experiment
def weird_rabi(
    qop,
    q,
    q_number,
    count,
    transition="ge",
    cal_traces=True,
):
    
    """Rabi where number of pulses is swept"""
    with dsl.acquire_loop_rt(
        count=count,
    ):
        with dsl.sweep(
            name=f"npulses_{q.uid}",
            parameter=SweepParameter(f"npulses_{q.uid}", q_number),
        ) as number_of_pulses:
            qop.prepare_state(q, transition[0])
            qop.x180(q, number_of_pulses)
            qop.measure(q, f"result/{q.uid}")
            qop.passive_reset(q)

        if cal_traces:
            with dsl.section(
                name=f"cal_states/{q.uid}",
            ):
                for state in transition:
                    qop.prepare_state(q, state)
                    qop.measure(q, f"cal_trace/{q.uid}/{state}")
                    qop.passive_reset(q)

In [11]:
weird_rabi(
    qop,
    qubits[0],
    q_number=np.linspace(0, 14, 15),    
    count=100,
    )

AttributeError: 'TunableTransmonOperations' object has no attribute 'transition_parameters'

In [12]:
qubits[0].transition_parameters()

('drive',
 {'amplitude_pi': 0.4,
  'amplitude_pi2': 0.8,
  'length': 5.1e-08,
  'pulse': {'function': 'drag', 'beta': 0.01, 'sigma': 0.21}})